
# Demystifying Approximate Bayesian Computation

## Brett Morris

### In this tutorial

We will write our own rejection sampling algorithm to approximate the posterior distributions for some fitting parameters.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import anderson_ksamp

First, let's generate a series of observations $y_\mathrm{obs}$, taken at times $x$. The observations will be drawn from one of two Gaussian distributions with a fixed standard deviation, separated by $3\sigma$ from one another.

In [ ]:
np.random.seed(42)

true_std = 1
true_mean1 = np.pi
true_mean2 = 3 * np.pi

x = np.linspace(0, 1, 250)

y_obs = np.concatenate([true_mean1 + true_std * np.random.randn(len(x)), 
                        true_mean2 + true_std * np.random.randn(len(x))])

plt.hist(y_obs, bins=50)
plt.xlabel('$y_\mathrm{obs}$', fontsize=16)
plt.show()

So how does one fit for the means and standard deviations of the bimodal distribution? One way is to use [Gaussian mixture models](https://dfm.io/posts/mixture-models/), but we're going to take a different approach. 

## Approximate Bayesian Computation

For this particular dataset, it's easy to construct a model $\mathcal{M}$ which reproduces the observations $y_\mathcal{obs}$ – the model is simply the concatenation of two normal distributions $\mathcal{M} \sim [\mathcal{N}(\mu_1, \sigma), \mathcal{N}(\mu_2, \sigma))]$. One way to *approximate* the posterior distributions of $\theta = \{\mu_1, \mu_2, \sigma\}$ would be to propose new parameters $\theta^*$, and only keep a running list of the parameter combinations which produce a simulated dataset $y_\mathrm{sim}$ which very closely reproduces the observations $y_\mathrm{obs}$. 

### Summary statistic: the Anderson-Darling statistic

In practice, this requires a *summary statistic*, which measures the "distance" between the simulated dataset $y_\mathrm{sim}$ and the observations $y_\mathrm{obs}$. In this example we need a metric which measures the probability that two randomly-drawn samples $y$ are drawn from the same distribution. One such metric is the [Anderson-Darling statistic](https://en.wikipedia.org/wiki/Anderson–Darling_test), which approaches a minimum near $A^2=-1.3$ for two sets $y$ that are drawn from indistinguishable distributions, and grows to $A^2 > 10^5$ for easily distinguishable distributions.

We can see how the Anderson-Darling statistic behaves in this simple example below: 

In [ ]:
n_samples = 10000

# Generaet a bimodal distribution
a = np.concatenate([np.random.randn(n_samples), 
                    3 + np.random.randn(n_samples//2)])

# Plot the bimodal distribution
fig, ax = plt.subplots(1, 2, figsize=(7, 3))
ax[0].hist(a, color='silver', range=[-4, 11], bins=50,
           lw=2, histtype='stepfilled')

# For a set of bimodal distributions with varing means: 
for mean in [0, 1.2, 5]: 
    # Generate a new bimodal distribution
    c = mean + np.concatenate([np.random.randn(n_samples), 
                               3 + np.random.randn(n_samples//2)])
    
    # Measure, plot the Anderson-darling statistic
    a2 = anderson_ksamp([a, c]).statistic
    ax[0].hist(c, histtype='step', range=[-4, 11], 
               bins=50, lw=2)

    ax[1].plot(mean, a2, 'o')

ax[0].set(xlabel='Samples', ylabel='Frequency')
ax[1].set(xlabel='Mean', ylabel='$A^2$')

fig.tight_layout()

In the figure above, we have a set of observations $y_\mathrm{obs}$ (left, gray) which we're comparing to the set of simulated observations $y_\mathrm{sim}$ (left, colors). The Anderson-Darling statistic $A^2$ is plotted for each pair of the observations and the simulations (right). You can see that the minimum of $A^2$ is near -1, and it grows very large when $y_\mathrm{obs}$ and $y_\mathrm{sim}$ distributions are significantly different.

### Rejection sampler

We're now have the ingredients we need to create a *rejection sampler*, which will follow this algorithm: 

  1. Perturb initial/previous parameters $\theta$ by a small amount to generate new trial parameters $\theta^*$
  
  2. If the trial parameters $\theta^*$ are drawn from within the prior, continue, else return to (1)
  
  3. Generate an example dataset $y_\mathrm{sim}$ using your model $\mathcal{M}$ 
  
  4. Compute _distance_ between the simulated and observed datasets $\rho(y_\mathrm{obs}, y_\mathrm{sim})$
  
  5. For some tolerance $h$, accept the step ($\theta^* = \theta$) if distance $\rho(y_\mathrm{obs}, y_\mathrm{sim}) \leq h$
  
  6. Return to step (1)

In [ ]:
def lnprior(theta): 
    """
    Define a prior probability, which simply requires 
    that -10 < mu_1, mu_2 < 20 and 0 < sigma < 10
    """
    mean1, mean2, std = theta
    
    if -10 < mean1 < 20 and -10 < mean2 < 20 and 0 < std < 10: 
        return 0
    return -np.inf

def propose_step(theta, scale): 
    """
    Propose new step: perturb the previous step
    by adding random-normal values to the previous step
    """
    return theta + scale * np.random.randn(len(theta))

def simulate_dataset(theta): 
    """
    Simulate a dataset by generating a bimodal distribution
    with means mu_1, mu_2 and standard deviation sigma
    """
    mean1, mean2, std = theta
    return np.concatenate([mean1 + std * np.random.randn(len(x)), 
                           mean2 + std * np.random.randn(len(x))])

def summary_stats(y_sim):
    """
    Compute the Anderson-Darling statistic as the distance
    metric between the simulated observations y_sim and the 
    observations y_obs
    """
    distance = anderson_ksamp([y_sim, y_obs]).statistic
    
    return distance

In [ ]:
def rejection_sampler(theta, h, n_steps, scale=0.1, quiet=False):
    """
    Follow algorithm written above for a simple rejection sampler. 
    """
    # Some bookkeeping variables:
    accepted_steps = 0
    total_steps = 0
    samples = np.zeros((n_steps, len(theta)))
    printed = set()

    while accepted_steps < n_steps: 

        # Make a simple "progress bar":
        if not quiet:
            if accepted_steps % 1000 == 0 and accepted_steps not in printed:
                printed.add(accepted_steps)
                print(f'Sample {accepted_steps} of {n_steps}')

        # Propose a new step:
        new_theta = propose_step(theta, scale)
        prior = lnprior(new_theta)

        # If proposed step is within prior: 
        if np.isfinite(prior): 

            # Generate a simulated dataset from new parameters
            y_sim = simulate_dataset(new_theta)

            # Compute distance between simulated dataset
            # and the observations
            distance = summary_stats(y_sim)
            total_steps += 1

            # If distance is less than tolerance `h`, accept step:
            if distance <= h: 
                theta = new_theta
                samples[accepted_steps, :] = new_theta
                accepted_steps += 1

    print(f'Acceptance rate: {accepted_steps/total_steps}')
    return samples

We can now run our rejection sampler for a given value of the tolerance $h$. 

In [ ]:
# Initial step parameters for the mean and std:
theta = [true_mean1, true_mean2, true_std] 

# Number of posterior samples to compute
n_steps = 5000


# `h` is the distance metric threshold for acceptance;
# try values of h between -0.5 and 5
h = 0

samples = rejection_sampler(theta, h, n_steps)

`samples` now contains `n_steps` approximate posterior samples. Let's make a corner plot which shows the results: 

In [ ]:
from corner import corner

corner(samples, truths=[true_mean1, true_mean2, true_std], 
       levels=[0.6], labels=['mean 1', 'mean 2', 'std'], 
       show_titles=True);

You can experiment with the above example by changing the values of from $h=-0.8$, for a more precise and more computationally expensive approximation to the posterior distribution, or to $h=10$ for a faster but less precise estimate of the posterior distribution. 

In practice, a significant fraction of your effort when applying ABC is spent balancing the computational expense of a small $h$ with the precision you need on your posterior approximation.

We can see how the posterior distribution for the standard deviation $\sigma$ varies as we vary $h$: 

In [ ]:
samples_i = []
h_range = [-0.5, 1, 5]

for h_i in h_range: 
    samples_i.append(rejection_sampler(theta, h_i, n_steps, quiet=True))

In [ ]:
for s_i, h_i in zip(samples_i, h_range): 
    plt.hist(s_i[:, 2], histtype='step', lw=2, 
             label=f"h={h_i}", density=True, 
             range=[0.5, 2], bins=50)
plt.legend()
plt.xlabel('std')
plt.axvline(true_std, ls='--', color='DodgerBlue')

In the plot above, you can see that the posterior distribution for the standard deviation is largest for the largest $h$, and converges to a narrower distribution centered on the correct value as $h$ decreases.